In [ ]:
!pip install matplotlib
!pip install pillow


In [ ]:
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.callbacks import EarlyStopping
from keras.layers import Average

import keras


from keras.optimizers import SGD
from keras.layers import Input, Conv2D, BatchNormalization, Activation
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import Callback
from IPython.display import clear_output
from keras.layers import MaxPooling2D, Flatten, Dense, Dropout
from keras import Model
from keras.layers import GlobalAveragePooling2D

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.applications.mobilenet import decode_predictions
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.vgg16 import VGG16
# from keras.applications.mobilenet import preprocess_input
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from keras.applications.inception_v3 import preprocess_input as preprocess_input_inc
from keras import optimizers

import pickle

# Import data

In [ ]:
!pwd

In [ ]:
!ls /opt/app-root/s3_home/uploads
# !unzip s3_home/uploads/buildings_data_smallest.zip

In [ ]:
# the complete dataset has 8 classes.
#the 'smaller' version 5 classes
#the 'smallest' version 3 classes


which_data = '_smaller' #choice between '', '_smaller', '_smallest'

### data preprocessed for vgg

In [ ]:
#I want to keep as many obs as possible in train, so I split the test in test and validation

#define generators
datagen_vgg_train = ImageDataGenerator(featurewise_center = False,
#                             rescale = 1/256,
                            preprocessing_function=preprocess_input_vgg)
datagen_vgg_val_test = ImageDataGenerator(featurewise_center = False,
#                             rescale = 1/256,
                            preprocessing_function=preprocess_input_vgg,
                            validation_split=0.4)

#version not processed
datagen_vgg_train_nop = ImageDataGenerator(rescale=1/256)
datagen_vgg_val_test_nop = ImageDataGenerator(rescale=1/256, validation_split=0.4)


#import data
train_in_vgg = datagen_vgg_train.flow_from_directory('Building_labeled_train_data'+str(which_data),
#                                       color_mode="rgb",
#                                        target_size = (128, 128),
#                                        batch_size=32,
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42)
train_in_vgg_nop = datagen_vgg_train_nop.flow_from_directory('Building_labeled_train_data'+str(which_data), seed=42, shuffle=True)

val_in_vgg = datagen_vgg_val_test.flow_from_directory('Building_labeled_test_data'+str(which_data),
#                                       color_mode="rgb",
#                                        target_size = (128, 128),
#                                        batch_size=32,
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42,
                                       subset='training') #this is because I used train_test split in the generator for val and test
val_in_vgg_nop = datagen_vgg_val_test_nop.flow_from_directory('Building_labeled_test_data'+str(which_data), seed=42, subset='training', shuffle=True)

test_in_vgg = datagen_vgg_val_test.flow_from_directory('Building_labeled_test_data'+str(which_data),
#                                       color_mode="rgb",
#                                       target_size = (128, 128),
#                                        batch_size=32,
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42,
                                       subset='validation') #this is because I used train_test split in the generator for val and test
test_in_vgg_nop = datagen_vgg_val_test_nop.flow_from_directory('Building_labeled_test_data'+str(which_data), seed=42, subset='validation', shuffle=True)

In [ ]:
X_train, y_train = train_in_vgg.next()
X_val, y_val = val_in_vgg.next()
X_test, y_test = test_in_vgg.next()

X_train_nop, y_train_nop = train_in_vgg_nop.next()
X_val_nop, y_val_nop = val_in_vgg_nop.next()
X_test_nop, y_test_nop = test_in_vgg_nop.next()

print ('X_train: ', X_train.shape,
       '\ny_train: ', y_train.shape,
       '\nX_val: ', X_val.shape,
       '\ny_val: ', y_val.shape,
       '\nX_test: ', X_test.shape,
       '\ny_test: ', y_test.shape)

#print an example
x,y = val_in_vgg.next()
x_nop, y_nop = val_in_vgg_nop.next()
for i in range(0,1):
    image = x[i] 
    image_nop = x_nop[i]
    plt.imshow(image_nop)
    plt.show()
    print('processed:')
    plt.imshow(image)
    plt.show()

In [ ]:
input_shape = X_train[0].shape #the input shape should not include batch size
n_classes = y_train[0].shape[0]

In [ ]:
input_shape

# Data augmentation

In [ ]:
train_datagen_vgg = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=70,
        horizontal_flip=True,
        preprocessing_function=preprocess_input_vgg)

val_test_datagen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                        validation_split = 0.4)


#import data
train_in_vgg_aug = train_datagen_vgg.flow_from_directory('Building_labeled_train_data'+str(which_data),
                                       class_mode="categorical",
                                       shuffle=True,
                                       seed=42)

val_in_vgg_aug = val_test_datagen_vgg.flow_from_directory('Building_labeled_test_data'+str(which_data),
                                       class_mode="categorical",
                                       shuffle=False,
                                       seed=42,
                                       subset='training')

test_in_vgg_aug = val_test_datagen_vgg.flow_from_directory('Building_labeled_test_data'+str(which_data),
                                       class_mode="categorical",
                                       shuffle=False,
                                       seed=42,
                                       subset='validation')


# train_in_inc_aug = train_datagen_inc.flow_from_directory('Building_labeled_train_data'+str(which_data),
#                                        class_mode="categorical",
#                                        shuffle=True,
#                                        seed=42)

# test_in_inc_aug = test_datagen_inc.flow_from_directory('Building_labeled_test_data'+str(which_data),
#                                        class_mode="categorical",
#                                        shuffle=True,
#                                        seed=42)



# Callbakcs

In [ ]:
class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.loss = []
        self.val_loss = []
        self.accuracy = []
        self.val_accuracy = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.accuracy.append(logs.get('accuracy'))
        self.val_accuracy.append(logs.get('val_accuracy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(20,5))
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.loss, label="loss")
        ax1.plot(self.x, self.val_loss, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.accuracy, label="accuracy")
        ax2.plot(self.x, self.val_accuracy, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot_learning = PlotLearning()

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=0.005, 
                           patience=15, verbose=1, mode='auto')

In [ ]:
reducelr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr = 0.001)

# simple model

In [ ]:
clear_session()


In [ ]:
def basic_cnn_model(input_shape):
    X_input = Input((input_shape))
    
    #conv
    X = Conv2D(16, (3,3), name='conv0')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2), name='max_pool0')(X)
    
    X = Conv2D(25, (6,6), name='conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2), name='max_pool1')(X)

    #dropout
    X = Dropout(0.5)(X)
    
    #rest
    X = Flatten()(X)
    
    #add one dense layer
    X = Dense(150, activation='relu', name='dense')(X)
    
    X = Dropout(0.5)(X)
    
    #final prediction
    X = Dense(n_classes, activation='softmax', name='final_dense')(X)
    
    model = Model(inputs = X_input, outputs = X,  name='basic_cnn')
    #here we are only building the model, that starting from X_input leads to (the last
    #X) through all the layers
    return model

In [ ]:
cnn_model = basic_cnn_model(input_shape)

In [ ]:
cnn_model.summary()

In [ ]:
#optimizers
from keras import optimizers
sgd = optimizers.SGD(learning_rate=0.001, decay=0.001)

sgd2 = optimizers.Adam()

In [ ]:
cnn_model.compile(optimizer=sgd, loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [ ]:
history = cnn_model.fit_generator(train_in_vgg, epochs=50, 
                                  validation_data=val_in_vgg, 
                                  callbacks=[early_stop, plot_learning, reducelr],
                                  use_multiprocessing=True)

In [ ]:
print('initial accuracy on train: ', cnn_model.history.history['accuracy'][0])
print('final accuracy on train: ', cnn_model.history.history['accuracy'][-1])

print('initial accuracy on val: ', cnn_model.history.history['val_accuracy'][0])
print('final accuracy on val: ', cnn_model.history.history['val_accuracy'][-1])

# transfer learning

In [ ]:
clear_session()

In [ ]:
# with open('s3_home/uploads/base_model_vgg128.pickle', 'rb') as f:
#     base_model_vgg128 = pickle.load(f)
    
with open('/opt/app-root/s3_home/uploads/base_model_vgg256.pickle', 'rb') as f:
    base_model_vgg256 = pickle.load(f)
    
with open('/opt/app-root/s3_home/uploads/base_model_inceptionv3_256.pickle', 'rb') as f:
    base_model_inception256 = pickle.load(f)

In [ ]:
base_model = base_model_vgg256

In [ ]:
base_model.summary()

In [ ]:
# add the end of the net

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
# x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
# # x = Dropout(0.2)(x)
# # x = Dense(256, activation='relu')(x)
# # x = Dropout(0.3)(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
# x = Dropout(0.4)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(n_classes, activation='softmax')(x)

In [ ]:
#I make 2 models here because I want to try different lr and then ensemble them
model_vgg1 = Model(inputs=base_model.input, outputs=predictions)
# model_vgg2 = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
#freeze the base model layers
for layer in base_model.layers[:18]:
    layer.trainable = False
    
for layer in base_model.layers[18:]:
    layer.trainable = True

In [ ]:
for i, layer in enumerate(model_vgg1.layers):
   print(i, layer.name, layer.trainable)

# for layer in model.layers:
#     print(layer, layer.trainable)

In [ ]:
#baseline accuracy:
1/n_classes

In [ ]:
#optimizers
from keras import optimizers
sgd = optimizers.SGD(learning_rate=0.001, decay=0.001)

sgd2 = optimizers.Adam()

In [ ]:
%timeit

#used sgd1 till 40 epochs, went up to 0.7-08 for tr and 0.5-0.6 for val

model_vgg1.compile(optimizer=sgd2, loss='categorical_crossentropy',
             metrics=['accuracy'])


vgg1_history = model_vgg1.fit_generator(train_in_vgg, 
                                         epochs = 20,
                                         validation_data=val_in_vgg, 
                                         callbacks=[early_stop, plot_learning, reducelr],
                                         validation_freq=1,
                                         use_multiprocessing=True
                                         )
        

## Check mislabeled examples

In [ ]:
import numpy as np

#prediction on validation set
pred_proba = model_vgg1.predict_generator(val_in_vgg)
pred = [np.argmax(x) for x in pred_proba]

#real values
real_y_proba = y_val
real_y = [np.argmax(x) for x in real_y_proba]

In [ ]:
#correctly labeled
labels = {0:'apt', 1:'office'}

for p, image, y, probs in zip(pred, X_val_nop, real_y, pred_proba):
    if p == y:
        print(f'{labels[y]} predicted as {labels[p]}')
        print(f'probabilities: apt {np.round(probs[0], 3)}, office {np.round(probs[1], 3)}')
        plt.imshow(image)
        plt.show()

In [ ]:
#mislabeled
labels = {0:'apt', 1:'office'}

for p, image, y, probs in zip(pred, X_val_nop, real_y, pred_proba):
    if p != y:
        print(f'{labels[y]} predicted as {labels[p]}')
        print(f'probabilities: apt {np.round(probs[0], 3)}, office {np.round(probs[1], 3)}')
        plt.imshow(image)
        plt.show()

In [ ]:
#make a model with same architecture but different optimizer... to use for ensemble

# %timeit

# model_vgg2.compile(optimizer=sgd2, loss='categorical_crossentropy',
#              metrics=['accuracy'])


# vgg2_history = model_vgg2.fit_generator(train_in_vgg, 
#                                          epochs=15,
#                                          validation_data=val_in_vgg, 
#                                          callbacks=[early_stop, plot_learning, reducelr],
#                                          validation_freq=1,
#                                          use_multiprocessing=True
#                                          )


## check misclassified examples

In [ ]:
#first model

print('initial accuracy on train: ', vgg1_history.history['accuracy'][0])
print('final accuracy on train: ', vgg1_history.history['accuracy'][-1])

print('initial accuracy on val: ', vgg1_history.history['val_accuracy'][0])
print('final accuracy on val: ', vgg1_history.history['val_accuracy'][-1])

In [ ]:
#second model

print('initial accuracy on train: ', vgg2_history.history['accuracy'][0])
print('final accuracy on train: ', vgg2_history.history['accuracy'][-1])

print('initial accuracy on val: ', vgg2_history.history['val_accuracy'][0])
print('final accuracy on val: ', vgg2_history.history['val_accuracy'][-1])

# Emseble models

In [ ]:
#test ensemble
# from keras.layers import Average

# def ensemble(models, model_input):
#     outputs = [model.outputs[0] for model in models]
#     y = Average()(outputs)
#     model = Model(model_input, y, name='ensemble')
#     return model

# ens_model = ensemble([model_vgg1, model_vgg2], test_in_vgg)

In [ ]:
# probabilities = model.predict_generator(test_in_aug)

filenames = test_in_vgg.filenames
nb_samples = len(filenames)

In [ ]:
# predict = model.predict_generator(test_in_aug,steps = nb_samples)
# predicted_class_indices=np.argmax(predict,axis=1)
# labels = test_in_aug.classes

loss1, acc1 = model_vgg1.evaluate_generator(test_in_vgg, steps=nb_samples, verbose=0)
loss2, acc2 = model_vgg2.evaluate_generator(test_in_vgg, steps=nb_samples, verbose=0)

In [ ]:
loss1, acc1, loss2, acc2

In [ ]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
    return num 
  


def calculate_ensemble_pred(model1, model2, test_data):
    pred1 = model1.predict_generator(test_data)
    pred2 = model2.predict_generator(test_data)
    pred1_classes = [np.argmax(x) for x in pred1]
    pred2_classes = [np.argmax(x) for x in pred2]
    
    votes = [(pred1_classes[x], pred2_classes[x]) for x in range(len(pred1))]
    final_votes = [most_frequent(x) for x in votes]
    return final_votes

In [ ]:
# pred1 = cnn_model.predict_generator(test_in_vgg)

In [ ]:
test = calculate_ensemble_pred(model_vgg1, model_vgg2, test_in_vgg)
len(test)

In [ ]:
# # Plot training & validation accuracy values
# plt.plot(mn_history.history['accuracy'])
# plt.plot(mn_history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.plot(mn_history.history['loss'])
# plt.plot(mn_history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# Ensemble the models

In [ ]:
#model 1: my cnn

model1 = cnn_model

In [ ]:
#model 2: the vgg

model2 = model

In [ ]:
#model 3: inceptionv3

model3 = 

In [ ]:
with open('s3_home/uploads/base_model_inceptionv3_256.pickle', 'rb') as f:
    base_model_inception = pickle.load(f)

# base_model_inception.summary()